# Leave-One-Out Cross-Validation and Rotations
*This notebook verifies that the Leave-One-Out-Cross-Validation (LOOCV) is sensitive to rotations of the regression problem*

## Import Dependencies

In [1]:
import numpy as np

## Generate Random Data

In [2]:
np.random.seed(0)
n, p = 25, 3
sigma = 0.1
X = np.random.random_sample((n, p))
beta = np.random.random_sample(p)
y = np.dot(X, beta) + np.random.normal(scale=sigma, size=n)
Gamma = np.diag(np.random.random_sample(p))

## Generate a Random Rotation Matrix

In [3]:
def generate_rotation_matrix(n):
    result = np.identity(n)
    for i in range(n-1):
        rotation = np.identity(n)
        theta = np.random.uniform()*2*np.pi
        rotation[i, i] = np.cos(theta)
        rotation[i, i+1] = -np.sin(theta)
        rotation[i+1, i] = np.sin(theta)
        rotation[i+1, i+1] = np.cos(theta)
        result = np.dot(result, rotation)
    return result
rotation_matrix = generate_rotation_matrix(n)

## Compute LOOCV

In [4]:
def compute_loocv(X, y, Gamma):
    A = np.dot(X.T, X) + np.dot(Gamma.T, Gamma)
    A_inv = np.linalg.inv(A)
    b_hat = np.dot(A_inv, np.dot(X.T, y))
    y_hat = np.dot(X, b_hat)
    h = np.array([np.dot(x_i, np.dot(A_inv, x_i)) for x_i in X])
    return np.sum(((y - y_hat) / (1 - h))**2) / len(y)

## Verify LOOCV is sensitive to rotations

In [5]:
loocv = compute_loocv(X, y, Gamma)
X_prime = np.dot(rotation_matrix, X)
y_prime = np.dot(rotation_matrix, y)
loocv_rotated = compute_loocv(X_prime, y_prime, Gamma)
print("%f != %f" % (loocv, loocv_rotated))

0.007926 != 0.008100
